In [1]:
import cv2
import numpy as np
import time
import random

# Inicializar a captura de vídeo da webcam (índice 1 para a primeira webcam)
cap = cv2.VideoCapture(1)

# Verificar se a captura foi inicializada corretamente
if not cap.isOpened():
    print("Erro ao abrir a webcam")
    exit()

# Função callback vazia para as trackbars
def nothing(x):
    pass

# Criar uma janela para as trackbars
cv2.namedWindow('Trackbars')

# Criar trackbars para ajustar os valores de HSV
cv2.createTrackbar('H Min', 'Trackbars', 0, 179, nothing)
cv2.createTrackbar('H Max', 'Trackbars', 179, 179, nothing)
cv2.createTrackbar('S Min', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('S Max', 'Trackbars', 255, 255, nothing)
cv2.createTrackbar('V Min', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('V Max', 'Trackbars', 255, 255, nothing)

# Inicializar variáveis para o cálculo do FPS
prev_time = 0
fps = 0

# Carregar a imagem transparente (assumindo que é uma imagem PNG com canal alfa)
FOLDER = 'pokemon_dataset/images/'
POKEMON = 'abra'
sprite = cv2.imread(FOLDER + POKEMON + '.png', cv2.IMREAD_UNCHANGED)

# Inicializar a posição do sprite
sprite_x = random.randint(0, 640 - sprite.shape[1])
sprite_y = random.randint(0, 480 - sprite.shape[0])
sprite_speed_x = random.choice([-5, 5])
sprite_speed_y = random.choice([-5, 5])

# Contador de pontos
score = 0
game_started = False

# Capturar uma imagem de fundo para subtração de fundo
print("Posicione sua mão fora da câmera e pressione 's' para capturar o fundo.")
while True:
    ret, background = cap.read()
    cv2.imshow('Captura de Fundo', background)
    if cv2.waitKey(1) & 0xFF == ord('s'):
        break

cv2.destroyWindow('Captura de Fundo')

while True:
    # Capturar frame por frame
    ret, frame = cap.read()

    # Se o frame foi capturado corretamente, ret será True
    if not ret:
        print("Falha ao capturar o frame")
        break

    # Subtrair o fundo do frame atual
    fg_mask = cv2.absdiff(background, frame)

    # Converter o frame para o espaço de cor HSV
    hsv = cv2.cvtColor(fg_mask, cv2.COLOR_BGR2HSV)

    # Obter os valores atuais das trackbars
    h_min = cv2.getTrackbarPos('H Min', 'Trackbars')
    h_max = cv2.getTrackbarPos('H Max', 'Trackbars')
    s_min = cv2.getTrackbarPos('S Min', 'Trackbars')
    s_max = cv2.getTrackbarPos('S Max', 'Trackbars')
    v_min = cv2.getTrackbarPos('V Min', 'Trackbars')
    v_max = cv2.getTrackbarPos('V Max', 'Trackbars')

    # Definir os limites para a cor da pele em HSV
    lower_skin = np.array([h_min, s_min, v_min], dtype=np.uint8)
    upper_skin = np.array([h_max, s_max, v_max], dtype=np.uint8)

    # Criar uma máscara para a cor da pele
    skin_mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Aplicar operações morfológicas para limpar a máscara
    kernel = np.ones((3, 3), np.uint8)
    skin_mask = cv2.erode(skin_mask, kernel, iterations=2)
    skin_mask = cv2.dilate(skin_mask, kernel, iterations=2)

    # Encontrar contornos
    contours, hierarchy = cv2.findContours(skin_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Inicializar a área do convex hull
    hull_area = 0

    # Desenhar o convex hull do maior contorno (que presumivelmente é a mão) na imagem original
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        hull = cv2.convexHull(largest_contour)
        cv2.drawContours(frame, [hull], -1, (0, 255, 0), 2)
        hull_area = cv2.contourArea(hull)

    # Verificar se o jogo foi iniciado
    if not game_started:
        # Exibir mensagem para iniciar o jogo
        cv2.putText(frame, "Aperte 'n' para iniciar o jogo", (50, 240), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow('Webcam - Convex Hull', frame)
        cv2.imshow('Webcam - Threshold', skin_mask)

        # Esperar pela tecla 'n' para iniciar o jogo
        key = cv2.waitKey(1)
        if key & 0xFF == ord('n'):
            game_started = True
        elif key & 0xFF == ord('q'):
            break
        continue

    # Atualizar a posição do sprite
    sprite_x += sprite_speed_x
    sprite_y += sprite_speed_y

    # Verificar colisão com as bordas da janela
    if sprite_x < 0 or sprite_x + sprite.shape[1] > frame.shape[1]:
        sprite_speed_x *= -1
    if sprite_y < 0 or sprite_y + sprite.shape[0] > frame.shape[0]:
        sprite_speed_y *= -1

    # Verificar colisão com o convex hull
    if contours:
        for point in hull:
            if sprite_x < point[0][0] < sprite_x + sprite.shape[1] and sprite_y < point[0][1] < sprite_y + sprite.shape[0]:
                score += 1
                # Resetar a posição do sprite
                sprite_x = random.randint(0, 640 - sprite.shape[1])
                sprite_y = random.randint(0, 480 - sprite.shape[0])
                sprite_speed_x = random.choice([-5, 5])
                sprite_speed_y = random.choice([-5, 5])
                break

    # Ajustar os índices do sprite para garantir que estejam dentro dos limites do frame
    sprite_x = max(0, min(sprite_x, frame.shape[1] - sprite.shape[1]))
    sprite_y = max(0, min(sprite_y, frame.shape[0] - sprite.shape[0]))

    # Desenhar o sprite na imagem
    sprite_overlay = sprite[:, :, :3]
    alpha_mask = sprite[:, :, 3] / 255.0
    for c in range(3):
        frame[sprite_y:sprite_y + sprite.shape[0], sprite_x:sprite_x + sprite.shape[1], c] = sprite_overlay[:, :, c] * alpha_mask + frame[sprite_y:sprite_y + sprite.shape[0], sprite_x:sprite_x + sprite.shape[1], c] * (1 - alpha_mask)

    # Calcular o FPS
    curr_time = time.time()
    fps = 1 / (curr_time - prev_time)
    prev_time = curr_time

    # Adicionar o texto do FPS na imagem
    cv2.putText(frame, f'FPS: {int(fps)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)

    # Adicionar o texto da área do convex hull na imagem
    cv2.putText(frame, f'Area: {int(hull_area)}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)

    # Adicionar o texto do score na imagem
    cv2.putText(frame, f'Score: {int(score)}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)

    # Exibir o frame com o convex hull, FPS, área e score
    cv2.imshow('Webcam - Convex Hull', frame)

    # Exibir o resultado da limiarização
    cv2.imshow('Webcam - Threshold', skin_mask)

    # Esperar pela tecla 'q' para sair do loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar a captura e fechar todas as janelas
cap.release()
cv2.destroyAllWindows()

Posicione sua mão fora da câmera e pressione 's' para capturar o fundo.
